In [4]:
import pandas as pd
import numpy as np
import glob
import os
#import datacompy as dc
#import fnmatch

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# merging the files
path = r'/content/drive/Shared drives/DS4A_2021/Team74/data/raw' # path varaiable for source location 
all_files = glob.glob(path + "/*.csv") #returns list of all joined files 
dtypes = {'jobsreported': np.int64,'originatinglenderlocationid': np.int64,'sbaofficecode': np.int64,'servicinglenderlocationid': np.int64} #Dtype dict
li = []   

for filename in all_files:
    df_file = pd.read_csv(filename,dtype=dtypes, index_col=None, header=0,low_memory=False)
    li.append(df_file)

df = pd.concat(li, axis=0, ignore_index=True)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (16,50) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (50) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df.head(3)

,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,LoanStatus,Term,SBAGuarantyPercentage,InitialApprovalAmount,CurrentApprovalAmount,UndisbursedAmount,FranchiseName,ServicingLenderLocationID,ServicingLenderName,ServicingLenderAddress,ServicingLenderCity,ServicingLenderState,ServicingLenderZip,RuralUrbanIndicator,HubzoneIndicator,LMIIndicator,BusinessAgeDescription,ProjectCity,ProjectCountyName,ProjectState,ProjectZip,CD,JobsReported,NAICSCode,Race,Ethnicity,UTILITIES_PROCEED,PAYROLL_PROCEED,MORTGAGE_INTEREST_PROCEED,RENT_PROCEED,REFINANCE_EIDL_PROCEED,HEALTH_CARE_PROCEED,DEBT_INTEREST_PROCEED,BusinessType,OriginatingLenderLocationID,OriginatingLender,OriginatingLenderCity,OriginatingLenderState,Gender,Veteran,NonProfit
0,5375617707,05/01/2020,101.0,PPP,NOT AVAILABLE,NaN,NaN,NaN,NaN,NaN,Exemption 4,24,100,148440.00,148440.00,0.0,NaN,9551.0,"Bank of America, National Association","100 N Tryon St, Ste 170",CHARLOTTE,NC,28202-4024,U,N,N,New Business or 2 years or less,NaN,NaN,NaN,NaN,NaN,12.0,339114.0,Unanswered,Unknown/NotStated,NaN,148440.00,NaN,NaN,NaN,NaN,NaN,NaN,9551.0,"Bank of America, National Association",CHARLOTTE,NC,Unanswered,Unanswered,NaN
1,9677497701,05/01/2020,464.0,PPP,NORTH CHARLESTON HOSPITALITY GROUP LLC,192 College Park Rd,Ladson,NaN,29456-3517,NaN,Exemption 4,24,100,140081.78,140081.78,0.0,NaN,19248.0,Synovus Bank,1148 Broadway,COLUMBUS,GA,31901-2429,U,N,N,Existing or more than 2 years old,Ladson,BERKELEY,SC,29456-3517,SC-01,3.0,NaN,Unanswered,Unknown/NotStated,NaN,140081.78,NaN,NaN,NaN,NaN,NaN,Limited Liability Company(LLC),19248.0,Synovus Bank,COLUMBUS,GA,Unanswered,Unanswered,NaN
2,9547167709,05/01/2020,464.0,PPP,Q AND J SERVICES LLC,301 Old Georgetown Road,Manning,NaN,29102-2734,04/20/2021,Paid in Full,24,100,136520.96,136520.96,0.0,NaN,19248.0,Synovus Bank,1148 Broadway,COLUMBUS,GA,31901-2429,U,Y,Y,Existing or more than 2 years old,Manning,CLARENDON,SC,29102-2734,SC-06,170.0,541990.0,Unanswered,Unknown/NotStated,NaN,136520.96,NaN,NaN,NaN,NaN,NaN,Limited Liability Company(LLC),19248.0,Synovus Bank,COLUMBUS,GA,Unanswered,Unanswered,NaN


In [5]:
# write merged dataframe to drive
df.to_csv (r'/content/drive/Shared drives/DS4A_2021/Team74/data/Stage/loan_files_stage.csv', index = None, header=True) 

In [7]:
# read merged file
df = pd.read_csv('/content/drive/Shared drives/DS4A_2021/Team74/data/Stage/loan_files_stage.csv',low_memory=False)


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (16,50) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
df.shape

(10697219, 51)

In [10]:
df.dtypes

LoanNumber                       int64
DateApproved                    object
SBAOfficeCode                  float64
ProcessingMethod                object
BorrowerName                    object
BorrowerAddress                 object
BorrowerCity                    object
BorrowerState                   object
BorrowerZip                     object
LoanStatusDate                  object
LoanStatus                      object
Term                             int64
SBAGuarantyPercentage            int64
InitialApprovalAmount          float64
CurrentApprovalAmount          float64
UndisbursedAmount              float64
FranchiseName                   object
ServicingLenderLocationID      float64
ServicingLenderName             object
ServicingLenderAddress          object
ServicingLenderCity             object
ServicingLenderState            object
ServicingLenderZip              object
RuralUrbanIndicator             object
HubzoneIndicator                object
LMIIndicator             

In [ ]:
df.shape

(964361, 51)

Convert Data type

In [ ]:
df['DateApproved'] = pd.to_datetime(df['DateApproved'])
df['LoanStatusDate'] = pd.to_datetime(df['LoanStatusDate']) 